In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('..')

In [ ]:
from generate_pytorch import init_model
model = init_model()

In [ ]:
from evaluate_pytorch import ImageGenerator

In [ ]:
ig = ImageGenerator(model, num_image_tiles=1, trunc_psi=0.6)

In [ ]:
latents = ig.gen_latents()
gen_noise = ig.gen_noise()

w_styles = ig.map_styles(ig.model.GAN.S, latents)

In [ ]:
a = ig.generate_regular(w_styles, gen_noise)

In [ ]:
import numpy as np
# Factorize semantics from weight.
weights = []
for block in model.GAN.G.blocks:
    weight = block.to_style1.weight.T
    weights.append(weight.detach().numpy())
weight = np.concatenate(weights, axis=1).astype(np.float32)
weight = weight / np.linalg.norm(weight, axis=0, keepdims=True)
eigen_values, eigen_vectors = np.linalg.eig(weight.dot(weight.T))
features = eigen_vectors
scales = np.sqrt(eigen_values)

In [ ]:
from torch import from_numpy
def shift(styles, feature, scale, factor):
    return styles + factor * scale * from_numpy(feature).expand(styles.size())

In [ ]:
feature_id = 10
filenames = []
for i in range(10):
    new_styles = shift(w_styles, features[feature_id], scales[feature_id], 0.1 * float(i))
    name = 'shift' + str(i)
    ig.sample_name = name
    filenames.append(name + '.jpg')
    a = ig.generate_regular(new_styles, gen_noise)

In [ ]:
import imageio
images = []
path = '/home/osboxes/projects/mynn/generators/results/faces/'
for filename in filenames:
    images.append(imageio.imread(path + filename))
imageio.mimsave(path + 'movie.gif', images)